# Import Section

In [74]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from time import time
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.qda import QDA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import NMF
import os
import csv

# Load Data & Labels

In [75]:
# road to file : should be changed if not executed on the original machine
FileRoot = '/home/ahmed/Documents/'

# Load train, validation, test as DafaFrames:
files = pd.read_table(os.path.join(FileRoot,'audio/train.txt'), sep='\s+', names=['file','label'])
validation = pd.read_table(os.path.join(FileRoot,'audio/dev.txt'), sep='\s+', names=['file','label'])
test = pd.read_table('test_files.txt', sep='\s+', names=['file'])

# labels of each audio file in training set and validation set
train_class = files['label'].as_matrix()
classes, train_labels = np.unique(train_class, return_inverse=True)

validation_class = validation['label'].as_matrix()
classes, validation_labels = np.unique(validation_class, return_inverse=True)

# concatenation of training and validation files to have new training set
training = pd.concat([files, validation], ignore_index=True)
allTrain = training['label'].as_matrix()
# extraction of labels of the new concatenated dataset
classes, allTrain = np.unique(allTrain, return_inverse=True)

<b> Note: </b> We have combined train files and test files to obtain ne training set.

### Use of Late Fusion For train and Test Datasets

<h4> Functions defined below do the combined and Late Fusion: We should specify the value of the "k" parameter.

In [76]:
def combinedFusion(df, labels, n_mfcc, k=1):
    """
    This function do both combined fusion and late fusion.
    This function is used with training and validation set.
    Late fusion is done if k=1
    this function returns X_combined, y (vector of labels) and the expansion factor of each observation
    """
    for i, afile in df.iterrows():
        y_,sr = librosa.load(os.path.join(FileRoot, afile.file), sr=None)
        mfcc = librosa.feature.mfcc(y=y_, n_fft=4096, hop_length=4096, n_mfcc=n_mfcc)
       
        if i==0:
            X = mfcc.T
            rows = X.shape[0]/k
            X = X[0:k*rows].reshape(rows,k,n_mfcc)
            X = np.mean(X,axis=1)
            y = labels[i]*np.ones(X.shape[0])
        else:
            tmp = mfcc.T
            tmp = tmp[0:k*(tmp.shape[0]/k)].reshape(tmp.shape[0]/k,k,n_mfcc)
            tmp = np.mean(tmp,axis=1)
            X = np.concatenate((X, tmp), axis=0)
            y = np.concatenate((y, labels[i]*np.ones(tmp.shape[0])), axis=0)
    y = y.astype(int)
    print "X combined fusion: ",X.shape
    print "y combined fusion: ",y.shape
    return X,y,rows

In [77]:
def combinedFusion_test(df, n_mfcc, k=1):
    """
    This function do both comnined or late fusion for test set.
    df: test dataframe
    n_mfcc: number of mfcc coeffecients
    k: number of lines used to calculate the mean
    if k=1: Late fusion is done
    it returns X_test after doing this operation and the expansion factor of each line of X_test.
    """
    for i, afile in df.iterrows():
        y_,sr = librosa.load(os.path.join(FileRoot, afile.file), sr=None)
        mfcc = librosa.feature.mfcc(y=y_, n_fft=4096, hop_length=4096, n_mfcc=n_mfcc)
        if i==0:
            X = mfcc.T
            rows = X.shape[0]/k
            X = X[0:k*rows].reshape(rows,k,n_mfcc)
            X = np.mean(X,axis=1)
        else:
            tmp = mfcc.T
            tmp = tmp[0:k*(tmp.shape[0]/k)].reshape(tmp.shape[0]/k,k,n_mfcc)
            tmp = np.mean(tmp,axis=1)
            X = np.concatenate((X, tmp), axis=0)
    print "X combined fusion: ",X.shape
    return X,rows

### Score Calculation: (Used For Validation):

In [78]:
def compute_pred_score(y_predict, y_labels):
    """
    comparing the predicted result with the true labels.
    The function returns the accuracy.
    """
    res = y_predict - y_labels
    score = 100*(y_labels.shape[0]-np.count_nonzero(res))/float(y_labels.shape[0])
    return score

## Apply Tranformations on Train and Test Datasets

### Tranformation on initial Train and Validation Datasets (this tranformation won't be used later):

In [79]:
# For local training and validation:
t0 = time()
n_mfcc = 40
X_train,y_train,rows_train = combinedFusion(files, train_labels, n_mfcc=n_mfcc, k=1)
X_val,y_val,rows_val = combinedFusion(validation, validation_labels, n_mfcc=n_mfcc, k=1)

fusion_time = (time() - t0)
print "done in %0.3fs" % fusion_time 

X combined fusion:  (68676, 40)
y combined fusion:  (68676,)
X combined fusion:  (34220, 40)
y combined fusion:  (34220,)
done in 69.496s


### Tranformation on Final Train Dataset (intial Train Dataset combined with validation) and Test Dataset :

In [80]:
# For the final test: take both training set and validation set as the training data
t0 = time()
n_mfcc = 40 #number of mfcc componnets
X,y,rows = combinedFusion(training, allTrain, n_mfcc=n_mfcc, k=1) # X : train data of combined intial train and validation
X_test, rows_test = combinedFusion_test(test, n_mfcc=n_mfcc, k=1) 
fusion_time = (time() - t0)
print "done in %0.3fs" % fusion_time 

X combined fusion:  (102896, 40)
y combined fusion:  (102896,)
X combined fusion:  (35164, 40)
done in 97.796s


## Save Vectors after applying Late Fusion into CSV Files (to be loaded another time without doing calculation again)

Save initial training and validation vectors after Late fusion: 

In [81]:
# Used for training and validation sets:
np.savetxt("X_training.csv", X_train, delimiter=',')
np.savetxt("y_training.txt", y_train, fmt='%d')
np.savetxt("X_validation.csv", X_val, delimiter=',')
np.savetxt("y_validation.txt", y_val, fmt='%d')

Save Final training and test vectors after Late fusion: 

In [82]:
# Used For Submission set:
np.savetxt("X_train_mixed.csv", X, delimiter=',')
np.savetxt("y_train_mixed.txt", y, fmt='%d')
np.savetxt("X_test.csv", X_test, delimiter=',')

## Load Saved Vectors :

In [83]:
# Used for training and validation sets:
X_train = pd.read_csv('X_training.csv', sep=',', header=None).values
X_val = pd.read_csv('X_validation.csv', sep=',', header=None).values
y_train = np.loadtxt('y_training.txt', dtype=np.int)
y_val = np.loadtxt('y_validation.txt', dtype=np.int)

In [84]:
# Used For submission:
X_train = pd.read_csv('X_train_mixed.csv', sep=',', header=None).values
y_train = np.loadtxt('y_train_mixed.txt', dtype=np.int)
X_test = pd.read_csv('X_test.csv', sep=',', header=None).values

In [85]:
def MajorityVote(res):
    """
    This Function computes prdicted label using majority rule.
    returns final vector of predictions
    """
    counts = np.bincount(res)
    result = np.argmax(counts)
    return result

def VoteFusion(y_fusioned, df, rows):
    """
    Before computing the vote Result after fusion, we do a reshape 
    operation to get exactly the same rows number as test files. 
    (i.e : number of lines = 290, n_columns: len(y_pred)/290)
    Then over each row, apply the function MajorityVote to get a majority vote result
    the function returns the final vector of predicted labels (290,)
    """
    y = y_fusioned.reshape((df.shape[0], rows))
    return np.apply_along_axis(MajorityVote, axis=1, arr=y)

# Training Phase:

### Normalize Data:

In [86]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Dimensionality Reduction

In [88]:
pca = PCA(n_components=n_mfcc, whiten=True, svd_solver='auto')
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

### Grid Search For MLP:

This portion of code take so much time to be finished (brute search).

In [89]:
"""param_grid = [
  {'hidden_layer_sizes': [50,100,150,200], 'alpha': np.logspace(-4,2,7), 'tol': np.logspace(-4,-1,4)}
 ]
mlp = MLPClassifier()
clf = GridSearchCV(mlp, param_grid)
clf.fit(X_train, y_train)
clf.best_params_
"""

"param_grid = [\n  {'hidden_layer_sizes': [50,100,150,200], 'alpha': np.logspace(-4,2,7), 'tol': np.logspace(-4,-1,4)}\n ]\nmlp = MLPClassifier()\nclf = GridSearchCV(mlp, param_grid)\nclf.fit(X_train, y_train)\nclf.best_params_\n"

### Fit Classifier:

In [90]:
t0 = time()

clf = MLPClassifier(hidden_layer_sizes=(300), tol=0.001,alpha=0.1)
clf.fit(X_train, y_train)

train_time = (time() - t0)
print "done in %0.3fs" % train_time 

done in 59.034s


## Prediction:

In [91]:
# For submission:
y_pred = clf.predict(X_test)
y_pred = y_pred.astype(int)
print y_pred.shape
res = VoteFusion(y_pred, test, rows)

(35164,)


## Save Predictions :

In [92]:
np.savetxt('y_pred.txt', res, fmt='%d')